In [ ]:
import os
import gzip

# Change directory to the folder where the .gz file is located
os.chdir('/content/drive/MyDrive/Data/')

# Specify the path to the .gz file
gz_file_path = 'amazon-meta.txt.gz'

# Extract the .gz file
with gzip.open(gz_file_path, 'rb') as gz_file:
    with open('amazon-meta.txt', 'wb') as out_file:
        out_file.write(gz_file.read())

In [ ]:
dataset_folder = '/content/drive/MyDrive/Data/'
file_name = 'amazon-meta.txt'
file_path = dataset_folder + file_name

In [ ]:
import numpy as np
import json
import re
import pandas as pd
from pandas import json_normalize

In [ ]:
def import_from_file(filename):
    with open(filename, 'r',encoding='utf-8', errors='ignore') as f:
        data = []
        entry = {}
        last_key = None
        for num,line in enumerate(f):
            if(num<3) :
                continue
            line = line.strip()
            if line.startswith('Id:'):
                if entry:
                    if "discontinued product" not in entry[last_key]:
                        data.append(entry)
                entry = {"Id":line[3:].strip()}

            elif line:
                if not line[0].isalpha() or ':' not in line:
                    entry[last_key] += ',' + line
                else:
                    key,value = line.split(':', 1)
                    entry[key.strip()] = value.strip()
                    last_key = key

        if entry:
            data.append(entry)
    return data

def post_Processing(filedata):
    for item in filedata:

        for key,value in item.items():
            if key == 'similar':
                count, *similar = value.split()
                item[key] = {"count":count, "ASIN ID":similar}
            elif key == 'categories':
                subentry = {}
                value = value.split(',')
                subentry['count'] = value[0]
                subentry['ASIN ID'] = value[1:]
                item[key] = subentry
            elif key == 'reviews':
                pass
                subentry = {}
                value = value.split(',')

                pattern = r'(\S+):\s*(\S+)'
                indexes = re.findall(pattern,value[0])
                for index in indexes:
                    subentry[index[0]] = index[1]

                reviewList = []
                for i in range(len(value[1:])):

                    currentReview = {}
                    currentReview["Review ID"] = i+1
                    currentReview["Date"] = re.split(r'\s+', value[1:][i])[0]
                    pattern = r'(\S+):\s*(\S+)'
                    indexes = re.findall(pattern,value[1:][i])
                    for index in indexes:
                        currentReview[index[0]] = index[1]

                    reviewList.append(currentReview)

                subentry['review list'] = reviewList
                item[key] = subentry

    return filedata



In [ ]:
data = import_from_file(file_path)
data = post_Processing(data)

In [ ]:
infile = open(dataset_folder+file_name[:-4]+'_filtered.json','w')
json.dump(data,infile,indent=4)